In [7]:
!pip3 install transformers==3.0.0

    100% |████████████████████████████████| 757kB 1.2MB/s ta 0:00:01
    100% |████████████████████████████████| 3.0MB 400kB/s ta 0:00:011
  Found existing installation: tokenizers 0.9.3
    Uninstalling tokenizers-0.9.3:
      Successfully uninstalled tokenizers-0.9.3
  Found existing installation: transformers 2.2.0
    Uninstalling transformers-2.2.0:
      Successfully uninstalled transformers-2.2.0


In [22]:
!pwd

/root/rohan/docvqa


In [1]:
import torch
from modeling_layoutlm import LayoutLMForTokenClassification
from transformers  import (
    BertConfig,
    BertTokenizer,
)
from utils_docvqa import (
    read_docvqa_examples,
    convert_examples_to_features)
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset
from transformers.data.processors.squad import SquadResult
from tqdm import tqdm
import numpy as np

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
MODEL_FOLDER = "./models/"
SAMPLE_DATA = "./models/sample_data.json"
LABELS = ["start","end"]
pad_token_label_id=-100
labels = ["start","end"]
max_seq_length = 512
max_query_length = 64
doc_stride = 128

In [3]:
device = torch.device("cpu")
torch.device(device)
model_class = LayoutLMForTokenClassification
config_class = BertConfig
tokenizer_class = BertTokenizer
config = config_class.from_pretrained(MODEL_FOLDER,num_labels=2,cache_dir=None)
model = model_class.from_pretrained(MODEL_FOLDER)
tokenizer = tokenizer_class.from_pretrained(MODEL_FOLDER,do_lower_case=True)

In [4]:
examples = read_docvqa_examples(SAMPLE_DATA, is_training=False)

In [5]:
features = convert_examples_to_features(
            examples=examples,
            label_list=labels,
            tokenizer=tokenizer,
            max_seq_length=max_seq_length,
            doc_stride=doc_stride,
            max_query_length=max_query_length,
            is_training=False,
            pad_token_label_id=pad_token_label_id)


INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 1000000000
INFO:tensorflow:example_index: 0
INFO:tensorflow:doc_span_index: 0
INFO:tensorflow:tokens: [CLS] what is the name of the contact person mentioned in letter ? [SEP] confidential . . . . r ##j ##rt pr approval date : 1 / 8 / 13 r ala ##s proposed release date : for response for release to : contact : p . carter route to initials pat ##e peggy carter ac ma ##ura payne david fish ##el tom gr ##is ##com diane barrow ##s ed black ##mer tow ru ##cker tr return to peggy carter , pr , 16 reynolds building 51 ##14 ##2 39 ##7 ##7 . . source : https : / / www . industry ##do ##cum ##ents . uc ##sf . ed ##u / doc ##s / x ##nb ##100 ##37 [SEP]
INFO:tensorflow:token_to_orig_map: 14:0 15:1 16:1 17:2 18:2 19:3 20:3 21:3 22:4 23:5 24:6 25:7 26:8 27:8 28:8 29:8 30:8 31:9 32:10 33:10 34:11 35:12 36:13 37:13 38:14 39:15 40:16 41:17 42:18 43:18 44:19 45:19 46:20 47:20 48:21 49:22 50:23 51:24 52:25 53:25 54:26 55:27 56:28 57:29 58:29 59:3

In [6]:
all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
all_bboxes = torch.tensor([f.boxes for f in features], dtype=torch.long)
all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)

eval_dataset = TensorDataset(
        all_input_ids, all_input_mask, all_segment_ids,all_bboxes,all_example_index)
eval_batch_size = 1
eval_sampler = (
        SequentialSampler(eval_dataset))

eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=eval_batch_size
    )

In [9]:
model.to(device)
all_results = []

def to_list(tensor):
    return tensor.detach().cpu().tolist()

for batch in tqdm(eval_dataloader, desc="Evaluating"):
    model.eval()
    batch = tuple(t.to(device) for t in batch)
    with torch.no_grad():
        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
        }
        inputs["bbox"] = batch[3]
        inputs["token_type_ids"] = (batch[2])
        outputs = model(**inputs)
    example_indices = batch[4]

    for i, example_index in enumerate(example_indices):
        eval_feature = features[example_index.item()]
        unique_id = int(eval_feature.unique_id)

        output = [to_list(output[i]) for output in outputs]

        start_logits, end_logits = output
        result = SquadResult(unique_id, start_logits, end_logits)
        all_results.append(result)
predictions_json = {}
assert len(all_results)==len(features)
for i in range(len(all_results)):
    start_index = np.argmax(all_results[i].start_logits)
    end_index = np.argmax(all_results[i].end_logits)
    pred_answer = features[i].tokens[start_index:end_index+1]
    pred_answer = ' '.join([x for x in pred_answer])
    pred_text = pred_answer.replace(' ##', '')
    question = features[i].tokens[1:features[i].tokens.index('[SEP]')]
    question_text = ' '.join([x for x in question])
    question_text = question_text.replace(' ##', '')
    print(question_text)
    print(pred_text)
   
        

Evaluating: 100%|██████████| 1/1 [00:00<00:00,  1.82it/s]

what is the name of the contact person mentioned in letter ?
p . carter
